In [211]:
import pandas as pd
import numpy as np
from pyranges import PyRanges


<h1> T2T CHM13

In [174]:
## Anotations
t2t_genes = pd.read_csv('../annotation_files/T2T-chm13-catLiftOffGenes.csv')
t2t_telomeres = pd.read_csv('../annotation_files/chm13v2.0_telomere.bed', sep='\t', names=['Chromosome','Start', 'End'])
t2t_sds = pd.read_csv('../annotation_files/chm13v2.0_SD.full.bed', sep='\t')
t2t_sds = t2t_sds[['#chr1', 'start1', 'end1']]
## Read and merge files 
anotation = pd.read_csv('../data/SGDP_anotation.csv', sep=',', encoding='latin-1')

ids_hg19 = pd.DataFrame()
ids_hg19['Sample'] = anotation['3-Illumina_ID']
ids_hg19['Region'] = anotation['10-Region']
t2t_sds.columns =['Chromosome','Start','End']
t2t_sds

/var/folders/05/4pfxtbrs5459ytz3j1jhcgt00000gn/T/ipykernel_22719/1889853910.py:2: DtypeWarning: Columns (23,36) have mixed types. Specify dtype option on import or set low_memory=False.
  t2t_genes = pd.read_csv('../annotation_files/T2T-chm13-catLiftOffGenes.csv')


,Chromosome,Start,End
0,chr1,1,13731
1,chr1,3,140066
2,chr1,16,13731
3,chr1,16,13731
4,chr1,16,198196
...,...,...,...
83511,chrY,62445192,62453929
83512,chrY,62445192,62454747
83513,chrY,62445192,62456950
83514,chrY,62445192,62457871


In [175]:
# is not working 
chm13 = pd.read_csv('../call_cnvr_chm13/individual_cnv_cnvr.txt', sep='\t')


/var/folders/05/4pfxtbrs5459ytz3j1jhcgt00000gn/T/ipykernel_22719/2170477776.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  chm13 = pd.read_csv('../call_cnvr_chm13/individual_cnv_cnvr.txt', sep='\t')


In [176]:
chm13 = chm13.dropna()
chm13['Length_cnvr'] = chm13['End'] - chm13['Start']


chm13 = chm13[chm13['Sample_ID'] == 'LP6005441-DNA_G07']

chm13 = chm13[~chm13['CNV_Value'].isin([2])]
chm13

,Chr,Start,End,CNVR_ID,Sample_ID,CNV_Start,CNV_End,CNV_Value,Length,Length_cnvr
1508570,22,0.0,12764469.0,CNVR_109274,LP6005441-DNA_G07,8285141,8289096,1,3956,12764469.0
1508571,22,0.0,12764469.0,CNVR_109274,LP6005441-DNA_G07,8292271,8294316,1,2046,12764469.0
1508572,22,0.0,12764469.0,CNVR_109274,LP6005441-DNA_G07,8294316,8297751,1,3436,12764469.0
1508573,22,0.0,12764469.0,CNVR_109274,LP6005441-DNA_G07,8297751,8303368,1,5618,12764469.0
1508574,22,0.0,12764469.0,CNVR_109274,LP6005441-DNA_G07,8304488,8306331,1,1844,12764469.0
...,...,...,...,...,...,...,...,...,...,...
14499930,1,248304532.0,248387328.0,CNVR_11357,LP6005441-DNA_G07,248364795,248366351,26,1557,82796.0
14499931,1,248304532.0,248387328.0,CNVR_11357,LP6005441-DNA_G07,248366351,248367622,27,1272,82796.0
14499932,1,248304532.0,248387328.0,CNVR_11357,LP6005441-DNA_G07,248367622,248368745,27,1124,82796.0
14499933,1,248304532.0,248387328.0,CNVR_11357,LP6005441-DNA_G07,248368745,248370339,42,1595,82796.0


In [177]:
## Keeping only samples that are on old cnvs 
chm13 = chm13.merge(ids_hg19, right_on=['Sample'], left_on=['Sample_ID'])

In [178]:
#REMOVE SAMPLES WITH ABERRATIONS

chm13#.value_counts(by=["Sample_ID"])
chm13.columns = ['Chromosome', 'Start', 'End', 'CNVR_ID', 'Sample_ID', 'CNV_start', 'CNV_end', 'CNV_Value','Length', 'Length_cnvr','Sample','Region']

In [179]:
#change to df when using telomeres and centromeres filtration
final_cnv = chm13.pivot_table(index=["Chromosome", "Start", "End", 'Length_cnvr'], 
                    columns='Sample', 
                    values='CNV_Value',
                        fill_value=2).reset_index()

final_cnv

Sample,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07
0,3,0.0,891494.0,891494.0,5.377358
1,3,900018.0,1000503.0,100485.0,6.000000
2,3,1575421.0,1594102.0,18681.0,19.000000
3,3,1758355.0,1945034.0,186679.0,31.000000
4,3,2268719.0,2287580.0,18861.0,3.000000
...,...,...,...,...,...
14867,9,148771046.0,149276995.0,505949.0,1.000000
14868,9,149278663.0,149484014.0,205351.0,1.000000
14869,9,149620306.0,149794372.0,174066.0,1.000000
14870,9,149803678.0,149871435.0,67757.0,1.000000


In [180]:
final_cnv_ranges = PyRanges(final_cnv)


/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [181]:
t2t_genes.columns = ['Chromosome', 'Start', 'End', 'name', 'score', 'strand',
       'thickStart', 'thickEnd', 'reserved', 'blockCount', 'blockSizes',
       'chromStarts', 'name2', 'cdsStartStat', 'cdsEndStat', 'exonFrames',
       'txId', 'type', 'geneName', 'geneType', 'sourceGene',
       'sourceTranscript', 'alignmentId', 'alternativeSourceTranscripts',
       'Paralogy', 'UnfilteredParalogy', 'collapsedGeneIds',
       'collapsedGeneNames', 'frameshift', 'exonAnnotationSupport',
       'intronAnnotationSupport', 'transcriptClass', 'transcriptModes',
       'validStart', 'validStop', 'properOrf', 'extra_paralog']

<h2> Genes

In [182]:
t2t_genes['Chromosome'] = t2t_genes['Chromosome'].str.replace('chr', '')
t2t_genes = PyRanges(t2t_genes)

t2t_telomeres['Chromosome'] = t2t_telomeres['Chromosome'].str.replace('chr', '')
t2t_telomeres = PyRanges(t2t_telomeres)

t2t_sds['Chromosome'] = t2t_sds['Chromosome'].str.replace('chr', '')
t2t_sds = PyRanges(t2t_sds)

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

In [183]:
#### t2t genes 
coordinates_genes = final_cnv_ranges.nearest(t2t_genes)
coordinates_genes = coordinates_genes.as_df()
coordinates_genes = coordinates_genes[coordinates_genes['Distance'] <1 ]

## T2T telomeres
coordinates_telomeres = final_cnv_ranges.nearest(t2t_telomeres)
coordinates_telomeres = coordinates_telomeres.as_df()
coordinates_telomeres = coordinates_telomeres[coordinates_telomeres['Distance'] <1 ]

#SDs 
coordinates_sds = final_cnv_ranges.nearest(t2t_sds)
coordinates_sds = coordinates_sds.as_df()
coordinates_sds = coordinates_sds[coordinates_sds['Distance'] <1 ]



,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07,Start_b,End_b,Distance
0,1,0,403567,403567.0,17.124031,1,13731,0
1,1,453948,466894,12946.0,1.000000,459773,460982,0
2,1,518975,605035,86060.0,3.000000,567686,568750,0
3,1,700181,782637,82456.0,1.000000,717376,721103,0
4,1,878563,973010,94447.0,4.025000,900919,924969,0
...,...,...,...,...,...,...,...,...
14867,21,0,45090682,45090682.0,139.091432,4,293068,0
14868,21,0,45090682,45090682.0,1.000000,4,293068,0
14869,22,15787747,51324926,35537179.0,4.877301,15712005,15938486,0
14870,22,0,12764469,12764469.0,137.948830,1,66809,0


<h2> T2T CNVnator

In [282]:
header_list = ['Type', 'Position', 'Lenght', 'RD', 'T-TEST-EVAL', 'GAUSSIAN-EVAL', 'T-TEST-EVAL(MIDDLE)','GAUSSIAN-EVAL(MIDDLE)', 'FRAQ-READ-QUALITY', 't', 't2']

t2t_data = pd.read_csv('/Users/luciabazan/t2t_LP6005441-DNA_G07_cnvs.tsv', sep='\t' , names=header_list)
t2t_data


t2t_data = t2t_data[t2t_data['T-TEST-EVAL'] <= 0.05]
t2t_data

cnvnator_data = t2t_data

#SEPARATE COORDINATES
coordinates = cnvnator_data['Position'].str.split(':', expand=True)
CHROMOSOME = coordinates[0]

split_position = coordinates[1].str.split('-', expand=True)
START = split_position[0]
STOP = split_position[1]

cnvnator_style= pd.DataFrame()
cnvnator_style['Chromosome'] = CHROMOSOME
cnvnator_style['Start'] = START
cnvnator_style['End'] = STOP
cnvnator_style['CNV_value'] = cnvnator_data['RD']


cnvnator_style['Start'] = cnvnator_style['Start'].astype(int)
cnvnator_style['End'] = cnvnator_style['End'].astype(int)
cnvnator_style['CNV_value'] = cnvnator_style['CNV_value'].astype(float)

#cnvnator_style['Type'] = cnvnator_data['Type']

cnvnator_style['CNV_value'] = cnvnator_style['CNV_value'] * 2

cnvnator_style['Chromosome'] = cnvnator_style['Chromosome'].str.replace('chr','')

t2t_cnvnator = cnvnator_style
t2t_cnvnator['Length_cnvr'] = t2t_cnvnator['End'] - t2t_cnvnator['Start']
t2t_cnvnator= PyRanges(t2t_cnvnator)
t2t_cnvnator

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


,Chromosome,Start,End,CNV_value,Length_cnvr
0,1,25001,45000,3.3208,19999
1,1,48001,78000,3.2608,29999
2,1,88001,96000,0.5038,7999
3,1,2133001,2162000,7.1520,28999
4,1,2193001,2223000,5.9894,29999
...,...,...,...,...,...
1938,Y,2253001,5216000,0.1908,2962999
1939,Y,5218001,11518000,0.0382,6299999
1940,Y,11522001,62337000,0.0086,50814999
1941,Y,62339001,62399000,1.2280,59999


In [283]:
#### t2t genes 
t2t_cnvnator_coordinates_genes = t2t_cnvnator.nearest(t2t_genes)
t2t_cnvnator_coordinates_genes = t2t_cnvnator_coordinates_genes.as_df()
t2t_cnvnator_coordinates_genes = t2t_cnvnator_coordinates_genes[t2t_cnvnator_coordinates_genes['Distance'] <1 ]

## T2T telomeres
t2t_cnvnator_coordinates_telomeres = t2t_cnvnator.nearest(t2t_telomeres)
t2t_cnvnator_coordinates_telomeres = t2t_cnvnator_coordinates_telomeres.as_df()
t2t_cnvnator_coordinates_telomeres = t2t_cnvnator_coordinates_telomeres[t2t_cnvnator_coordinates_telomeres['Distance'] <1 ]

#SDs 
t2t_cnvnator_coordinates_sds = t2t_cnvnator.nearest(t2t_sds)
t2t_cnvnator_coordinates_sds = t2t_cnvnator_coordinates_sds.as_df()
t2t_cnvnator_coordinates_sds = t2t_cnvnator_coordinates_sds[t2t_cnvnator_coordinates_sds['Distance'] <1 ]



<h1> GRCh37

In [329]:
## Anotations
grch37_genes = pd.read_csv('../annotation_files/grch37-genes.csv')
grch37_telomeres = pd.read_csv('../annotation_files/grch37-telomers-centromers.csv')
grch37_sds = pd.read_csv('../annotation_files/grch37-sds.csv')
#grch37_sds = grch37_sds[['#chr1', 'start1', 'end1']]


## Read and merge files 
anotation = pd.read_csv('../data/SGDP_anotation.csv', sep=',', encoding='latin-1')

ids_hg19 = pd.DataFrame()
ids_hg19['Sample'] = anotation['3-Illumina_ID']
ids_hg19['Region'] = anotation['10-Region']


#t2t_sds.columns =['Chromosome','Start','End']
#t2t_sds
grch37_genes.columns = ['#"bin"', 'name', 'Chromosome', 'strand', 'Start', 'End', 'cdsStart',
       'cdsEnd', 'exonCount', 'exonStarts', 'exonEnds', 'score', 'name2',
       'cdsStartStat', 'cdsEndStat', 'exonFrames'] 
grch37_telomeres = grch37_telomeres[grch37_telomeres['type'] == 'telomere']
grch37_sds.columns = ['#"bin"', 'Chromosome', 'Start', 'End', 'name', 'score', 'strand',
       'otherChrom', 'otherStart', 'otherEnd', 'otherSize', 'uid',
       'posBasesHit', 'testResult', 'verdict', 'chits', 'ccov', 'alignfile',
       'alignL', 'indelN', 'indelS', 'alignB', 'matchB', 'mismatchB',
       'transitionsB', 'transversionsB', 'fracMatch', 'fracMatchIndel', 'jcK',
       'k2K']

In [330]:
data = pd.read_csv("../call_cnvr_cnvnator/individual_cnv_cnvr.txt", sep='\t')
data.columns = ['Chromosome', 'Start', 'End','CNVR','Sample_ID','CNV_Start', 'CNV_End', 'RD', 'Lenght']
data = data.dropna()#.describe()
data['Length_cnvr'] = data['End'] - data['Start'] 
sample_old = data[data['Sample_ID'] == 'LP6005441-DNA_G07']
#sample_old = old_assembly[old_assembly['Sample_ID'] == 'LP6005441-DNA_B10']
#sample_old = sample_old[['Chromosome', 'Start', 'End', 'RD']]
#sample_old['RD'] = sample_old['RD'] * 2
#sample_old['Chromosome'] = str('chr')+sample_old['Chromosome']
    #sample_old.to_csv('LP6005441-DNA_B10_grch37.bed', index=False, header=False, sep='\t')
    #list(sample_old.columns)

#sample_old

sample_old = sample_old[~sample_old['RD'].isin([2])]
#sample_old.columns = ['Chromosome', 'Start', 'End', 'RD']
#sample_old['assembly'] = 'GRCh37'
sample_old

,Chromosome,Start,End,CNVR,Sample_ID,CNV_Start,CNV_End,RD,Lenght,Length_cnvr
51663,1,176101.0,355800.0,CNVR_1,LP6005441-DNA_G07,234001,238300,4,4300.0,179699.0
51664,1,176101.0,355800.0,CNVR_1,LP6005441-DNA_G07,267101,317800,0,50700.0,179699.0
51665,1,449901.0,564900.0,CNVR_2,LP6005441-DNA_G07,471301,521400,0,50100.0,114999.0
51666,1,1007801.0,1015200.0,CNVR_5,LP6005441-DNA_G07,1010801,1014700,0,3900.0,7399.0
51667,1,3995301.0,4237400.0,CNVR_15,LP6005441-DNA_G07,4124401,4127800,1,3400.0,242099.0
...,...,...,...,...,...,...,...,...,...,...
52386,9,137097701.0,138417800.0,CNVR_4724,LP6005441-DNA_G07,138194201,138197800,1,3600.0,1320099.0
52387,9,137097701.0,138417800.0,CNVR_4724,LP6005441-DNA_G07,138214101,138217600,1,3500.0,1320099.0
52388,9,138418601.0,138480200.0,CNVR_4725,LP6005441-DNA_G07,138479101,138480200,0,1100.0,61599.0
52389,9,139663001.0,139715700.0,CNVR_4729,LP6005441-DNA_G07,139663601,139665300,0,1700.0,52699.0


In [331]:
###### Keeping only samples that are on old cnvs 
sample_old = sample_old.merge(ids_hg19, right_on=['Sample'], left_on=['Sample_ID'])

#REMOVE SAMPLES WITH ABERRATIONS
sample_old.columns = ['Chromosome', 'Start', 'End', 'CNVR_ID', 'Sample_iD', 'CNV_start', 'CNV_end', 'CNV_Value','Length', 'Length_cnvr','Sample','Region']


In [332]:
#change to df when using telomeres and centromeres filtration
final_cnv_grch37 = sample_old.pivot_table(index=["Chromosome", "Start", "End", 'Length_cnvr'], 
                    columns='Sample', 
                    values='CNV_Value',
                        fill_value=2).reset_index()


final_cnv_grch37

Sample,Chromosome,Start,End,Length_cnvr,LP6005441-DNA_G07
0,1,176101.0,355800.0,179699.0,2.0
1,1,449901.0,564900.0,114999.0,0.0
2,1,1007801.0,1015200.0,7399.0,0.0
3,1,3995301.0,4237400.0,242099.0,1.0
4,1,8360201.0,8364200.0,3999.0,0.0
...,...,...,...,...,...
412,9,136453701.0,136980500.0,526799.0,0.0
413,9,137097701.0,138417800.0,1320099.0,1.0
414,9,138418601.0,138480200.0,61599.0,0.0
415,9,139663001.0,139715700.0,52699.0,0.0


In [333]:
final_cnv_ranges_grch37 = PyRanges(final_cnv_grch37)
#final_cnv_grch37

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


<h2> Genes

In [334]:
grch37_genes['Chromosome'] = grch37_genes['Chromosome'].str.replace('chr', '')
grch37_genes_coordinates = PyRanges(grch37_genes)

grch37_telomeres['Chromosome'] = grch37_telomeres['Chromosome'].str.replace('chr', '')
grch37_telomeres = PyRanges(grch37_telomeres)

grch37_sds['Chromosome'] = grch37_sds['Chromosome'].str.replace('chr', '')
grch37_sds = PyRanges(grch37_sds)

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default

In [335]:
#Genes
grch37_coordinates_genes = final_cnv_ranges_grch37.nearest(grch37_genes_coordinates)
grch37_coordinates_genes = grch37_coordinates_genes.as_df()
grch37_coordinates_genes = grch37_coordinates_genes[grch37_coordinates_genes['Distance'] <1 ]
#grch37_coordinates_genes

#Telomeres
grch37_coordinates_telomeres = final_cnv_ranges_grch37.nearest(grch37_telomeres)
grch37_coordinates_telomeres = grch37_coordinates_telomeres.as_df()
grch37_coordinates_telomeres = grch37_coordinates_telomeres[grch37_coordinates_telomeres['Distance'] <1 ]

#SDs
grch37_coordinates_sds = final_cnv_ranges_grch37.nearest(grch37_sds)
grch37_coordinates_sds = grch37_coordinates_sds.as_df()
grch37_coordinates_sds = grch37_coordinates_sds[grch37_coordinates_sds['Distance'] <1 ]

In [336]:
grch37_genes = len(grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates())
grch37_genes

301

<h2> Peter Sudmant data 

In [337]:
sudmant = pd.read_csv('/Users/luciabazan/Downloads/sudmant_data.csv')
sudmant = sudmant[['contig','start','end','AMR_Maya_HGDP00855_F']]
sudmant.columns =['Chromosome', 'Start', 'End', 'CNV_value']


#sudmant['Chromosome'] = sudmant['Chromosome'].str.replace('chr','hs')
n_uncalled = np.sum(sudmant==-1,1)
sudmant = sudmant[n_uncalled==0]

sudmant['Length_cnvr'] = sudmant['End'] - sudmant['Start'] 
sudmant = sudmant[~sudmant['CNV_value'].isin([2])]


sudmant['Chromosome'] = sudmant['Chromosome'].str.replace('chr', '')
sudmant = PyRanges(sudmant)
sudmant

/opt/anaconda3/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


,Chromosome,Start,End,CNV_value,Length_cnvr
0,1,12048,177033,22,164985
1,1,12048,89237,10,77189
2,1,228416,267160,24,38744
3,1,471226,471326,10,100
4,1,318047,471226,21,153179
...,...,...,...,...,...
4476,X,75525579,75617789,3,92210
4477,X,84105273,84119490,3,14217
4478,X,134749333,134807363,3,58030
4479,X,29268810,29271507,3,2697


In [338]:
#Genes
peter_coordinates_genes = sudmant.nearest(grch37_genes_coordinates)
peter_coordinates_genes = peter_coordinates_genes.as_df()
peter_coordinates_genes = peter_coordinates_genes[peter_coordinates_genes['Distance'] <1 ]
#grch37_coordinates_genes

#Telomeres
peter_coordinates_telomeres = sudmant.nearest(grch37_telomeres)
peter_coordinates_telomeres = peter_coordinates_telomeres.as_df()
peter_coordinates_telomeres = peter_coordinates_telomeres[peter_coordinates_telomeres['Distance'] <1 ]

#SDs
peter_coordinates_sds = sudmant.nearest(grch37_sds)
peter_coordinates_sds = peter_coordinates_sds.as_df()
peter_coordinates_sds = peter_coordinates_sds[peter_coordinates_sds['Distance'] <1 ]

GRCh37 Duplications

In [339]:
d = {'Data': ['GRCh37', '  Genes','  Telomeres','  SDs','T2T-CHM13', '  Genes','  Telomeres','  SDs'
             ], 
     'Calls': [len(final_cnv_grch37), 
               len(grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(grch37_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(grch37_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),  
               len(t2t_cnvnator), 
               len(t2t_cnvnator_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(t2t_cnvnator_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(t2t_cnvnator_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())
              ], 
     'Mean size(Bp)':[final_cnv_grch37['Length_cnvr'].mean().round(), 
                      grch37_coordinates_genes['Length_cnvr'].mean().round(), 
                      grch37_coordinates_telomeres['Length_cnvr'].mean().round(), 
                      grch37_coordinates_sds['Length_cnvr'].mean().round(), 
                      t2t_cnvnator.as_df()['Length_cnvr'].mean().round(),
                      t2t_cnvnator_coordinates_genes['Length_cnvr'].mean().round(),
                      t2t_cnvnator_coordinates_telomeres['Length_cnvr'].mean().round(), 
                      t2t_cnvnator_coordinates_sds['Length_cnvr'].mean().round()],
     'Literature':[len(sudmant), 
                  len(peter_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                  len(peter_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(peter_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(final_cnv_ranges), 
                  len(coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                  len(coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()), ], 
     'Mean size(Bp) Literature':[sudmant.as_df()['Length_cnvr'].mean().round(), 
                      peter_coordinates_genes['Length_cnvr'].mean().round(), 
                      peter_coordinates_telomeres['Length_cnvr'].mean(), 
                      peter_coordinates_sds['Length_cnvr'].mean().round(), 
                      final_cnv['Length_cnvr'].mean().round(),
                      coordinates_genes['Length_cnvr'].mean().round(),
                      coordinates_telomeres['Length_cnvr'].mean().round(), 
                      coordinates_sds['Length_cnvr'].mean().round()]
    
    }
df = pd.DataFrame(data=d)
df

,Data,Calls,Mean size(Bp),Literature,Mean size(Bp) Literature
0,GRCh37,417,160454.0,4481,32000.0
1,Genes,301,211600.0,3179,39641.0
2,Telomeres,6,164382.0,0,NaN
3,SDs,163,310071.0,3849,35872.0
4,T2T-CHM13,1943,98033.0,14872,99326.0
5,Genes,433,207664.0,12264,108634.0
6,Telomeres,7,127570.0,42,8181399.0
7,SDs,630,158253.0,3374,268004.0


In [278]:
d = {'Data': ['GRCh37', '  Genes','  Telomeres','  SDs','T2T-CHM13', '  Genes','  Telomeres','  SDs'], 
     'Total': [len(final_cnv_grch37), 
               len(grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(grch37_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(grch37_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),  
               len(final_cnv), 
               len(coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
               len(coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates())], 
     'Mean size(Bp)':[final_cnv_grch37['Length_cnvr'].mean().round(), 
                      grch37_coordinates_genes['Length_cnvr'].mean().round(), 
                      grch37_coordinates_telomeres['Length_cnvr'].mean().round(), 
                      grch37_coordinates_sds['Length_cnvr'].mean().round(), 
                      coordinates_genes['Length_cnvr'].mean().round(),
                      final_cnv['Length_cnvr'].mean().round(),
                      coordinates_telomeres['Length_cnvr'].mean().round(), 
                      coordinates_sds['Length_cnvr'].mean().round()],
     'Deletions':[len(grch37_deletions), 
                  len(deletions_grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                  len(deletions_grch37_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(deletions_grch37_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(t2t_deletions), 
                  len(deletions_t2t_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                  len(deletions_t2t_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                  len(deletions_t2t_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()), ], 
     'Duplications':[len(grch37_duplications), 
                     len(duplications_grch37_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                     len(duplications_grch37_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                     len(duplications_grch37_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()),
                     len(t2t_duplications), 
                     len(duplications_t2t_coordinates_genes[['Chromosome','Start', 'End']].drop_duplicates()),
                     len(duplications_t2t_coordinates_telomeres[['Chromosome','Start', 'End']].drop_duplicates()), 
                     len(duplications_t2t_coordinates_sds[['Chromosome','Start', 'End']].drop_duplicates()) ]
    
    }
df = pd.DataFrame(data=d)
df

,Data,Total,Mean size(Bp),Deletions,Duplications
0,GRCh37,417,160454.0,417,417
1,Genes,301,211600.0,301,301
2,Telomeres,6,164382.0,6,6
3,SDs,163,310071.0,163,163
4,T2T-CHM13,14872,108634.0,14872,14872
5,Genes,12264,99326.0,12264,12264
6,Telomeres,42,8181399.0,42,42
7,SDs,3374,268004.0,3374,3374


In [280]:
final_cnv_grch37['Length_cnvr'].mean().round()

160454.0

In [279]:
t2t_cnvnator['Length_cnvr'].mean().round()

AttributeError: ('PyRanges object has no attribute', 'mean')

In [300]:
t2t_cnvnator.as_df()['Length_cnvr']

0          19999
1          29999
2           7999
3          28999
4          29999
          ...   
1938     2962999
1939     6299999
1940    50814999
1941       59999
1942       54999
Name: Length_cnvr, Length: 1943, dtype: int64

In [310]:
peter_coordinates_telomeres

,Chromosome,Start,End,CNV_value,Length_cnvr,"#""bin""",Start_b,End_b,ix,n,size,type,bridge,Distance
